In [1]:
#import statements
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.enableHiveSupport().appName('AmazonReviewRcommender').getOrCreate()
sc = spark.sparkContext

In [15]:
#check to see if thata is available in HDFS
!hdfs dfs -ls /user/hongyiwderrick/project

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
Found 3 items
-rw-r--r--   3 hongyiwderrick hongyiwderrick           0 2022-04-27 19:14 /user/hongyiwderrick/project/kcore_5.csv
-rw-r--r--   3 hongyiwderrick hongyiwderrick 32072979001 2022-04-27 19:21 /user/hongyiwderrick/project/kcore_5.json
-rw-r--r--   3 hongyiwderrick hongyiwderrick 10544467811 2022-04-27 19:29 /user/hongyiwderrick/project/metadata.json


### Load kcore_5 data

In [12]:
# Load metadata.json
df_kcore = spark.read.json("/user/hongyiwderrick/project/kcore_5.json")

In [14]:
df_kcore.show(2)

+----------+-------+-------+--------------------+-----------+--------------+------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+------------+--------------------+--------------+
|0000013714| [0, 0]|    4.0|We use this type ...| 12 3, 2013| ACNGUPJ3A3TM9|         GCM|         Nice Hymnal|    1386028800|
|0000013714| [2, 3]|    5.0|I bought this for...|09 13, 2009|A2SUAM1J3GNN3B| J. McDonald|Heavenly Highway ...|    1252800000|
+----------+-------+-------+--------------------+-----------+--------------+------------+--------------------+--------------+
only showing top 2 rows



In [23]:
df_kcore.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



### Load meta data

In [13]:
# Load metadata.json
df_meta = spark.read.json("/user/hongyiwderrick/project/metadata.json")

In [7]:
df_meta.show(2)

+---------------+----------+-----+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|_corrupt_record|      asin|brand|          categories|         description|               imUrl|price|             related|           salesRank|               title|
+---------------+----------+-----+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|           null|0001048791| null|           [[Books]]|                null|http://ecx.images...| null|                null|[,,,,, 6334800,,,...|The Crucible: Per...|
|           null|0000143561| null|[[Movies & TV, Mo...|3Pack DVD set - I...|http://g-ecx.imag...|12.99|[, [B0036FO6SI, B...|[,,,,,,,,,,,,,,,,...|Everyday Italian ...|
+---------------+----------+-----+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------

In [8]:
df_meta.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- imUrl: string (nullable = true)
 |-- price: double (nullable = true)
 |-- related: struct (nullable = true)
 |    |-- also_bought: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- also_viewed: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- bought_together: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- buy_after_viewing: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- salesRank: struct (nullable = true)
 |    |-- Appliances: long (nullable = true)
 |    |-- Arts, Crafts & Sewing: long (nullable = true)
 |    |-- Automotive: long

### Write to Hive table

In [16]:
from pyspark.sql import HiveContext
hive_context = HiveContext(sc)

In [17]:
hive_context.sql("use hongyiwderrick");
hive_context.sql("show tables").show();

+--------------+--------------------+-----------+
|      database|           tableName|isTemporary|
+--------------+--------------------+-----------+
|hongyiwderrick|   business_licenses|      false|
|hongyiwderrick|      chicago_crimes|      false|
|hongyiwderrick|      facility_types|      false|
|hongyiwderrick|     foodinspections|      false|
|hongyiwderrick|foodinspections_p...|      false|
+--------------+--------------------+-----------+



In [21]:
hive_context.sql("drop table if exists kcore_5")

DataFrame[]

In [22]:
df_kcore.write.mode('overwrite').saveAsTable('kcore_5')

In [24]:
hive_context.sql("select count(*) from kcore_5").show()

+--------+
|count(1)|
+--------+
|41135700|
+--------+



In [26]:
hive_context.sql("drop table if exists metadata")

DataFrame[]

In [ ]:
df_meta.write.mode('overwrite').saveAsTable('metadata')

Py4JJavaError: An error occurred while calling o315.saveAsTable.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.datasources.DataSource.writeAndRead(DataSource.scala:502)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.saveDataIntoTable(createDataSourceTables.scala:217)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.run(createDataSourceTables.scala:176)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	a

In [ ]:
hive_context.sql("select count(*) from metadata").show()